In [1]:
# These packages are not included in the MomentumED package. 
# Use the following line to add them:
# using Pkg; Pkg.add("CairoMakie"); Pkg.add("QuadGK"); Pkg.add("ClassicalOrthogonalPolynomials")

# Import the momentum-conserved exact diagonalization package
using MomentumED
include("Landau level triangular.jl")
using .LLT

using CairoMakie # for plotting
CairoMakie.activate!()

# Plot the energy spectrum
function plot_ed_spectrum()
    fig = Figure();
    ax = Axis(fig[1, 1];
        xlabel = "$(Gk[2])k1+k2",
        ylabel = "Energy per unit cell (W₀ = e²/ϵl)"
    )
    ax_top = Axis(fig[1, 1];
        xlabel = "momentum block number",
        xaxisposition = :top
    )
    top_ticks = ([], [])
    hidespines!(ax_top)
    hidexdecorations!(ax_top; label = false, ticklabels = false)
    hideydecorations!(ax_top)
    linkxaxes!(ax, ax_top)

    # Plot energy levels for each momentum block
    for i in 1:length(blocks)
        x = Gk[2] * block_k1[i] + block_k2[i]
        push!(top_ticks[1], x)
        push!(top_ticks[2], string(i))
        for e in energies[i]
            scatter!(ax, x, e/Nk/LLT.W0, color = :blue, marker=:hline)
        end
    end
    ax_top.xticks = top_ticks
    fig
end

plot_ed_spectrum (generic function with 1 method)

In [2]:
# Define k-mesh for bilayer system (4×3 mesh, Nk=12) in triangular lattice
k_list = [0 1 2 3 0 1 2 3 0 1 2 3;
          0 0 0 0 1 1 1 1 2 2 2 2]
Nk = 12
Gk = (4, 3)  # Grid dimensions


# number of electrons in each layer
Ne1 = 6
Ne2 = 6

# Number of layers (components) for bilayer system
Nc_conserve = 2

2

In [ ]:
# Interaction parameters

# Coulomb
LLT.W0 = 1.0                   # Interaction strength
LLT.D_l = 10.0                  # Screening length D/l
LLT.d_l = 0.1                  # Inter-layer distance d/l
# compute the pseudo-potential components
intra_PP = LLT.pseudo_potential_decomposition.(0:15, LLT.V_Coulomb_bilayer; SameLayer = true)
inter_PP = LLT.pseudo_potential_decomposition.(0:15, LLT.V_Coulomb_bilayer; SameLayer = false)
@show intra_PP
@show inter_PP

# Haldane
# LLT.V_intra = [0.0; 0.8; 0.0; 0.4]          # Intralayer Haldane pseudo-potential in unit of W0
# LLT.V_inter = [1.5; 0.7; 0.5; 0.4]          # Interlayer Haldane pseudo-potential in unit of W0

# or use Coulomb interaction with a cutoff in m
LLT.V_intra = copy(intra_PP)
LLT.V_inter = copy(inter_PP)

# choose a linear mixing between Haldane and Coulomb interaction
LLT.mix = 1                  # mix * Haldane + (1-mix) * Coulomb

intra_PP = [1.6347066215405632, 0.7493532823751158, 0.5286565941067406, 0.41872505368204593, 0.3503224973226579, 0.3026782860863603, 0.2671381550406759, 0.2393751098165452, 0.21695533203249928, 0.19839158716928695, 0.18271725829183577, 0.16927336306332738, 0.15759318809374134, 0.1473357319489056, 0.13824529565043234, 0.13012589008205516]
inter_PP = [1.538982326541291, 0.7473021108429173, 0.5281101963919731, 0.4184538927733024, 0.3501548452430184, 0.3025623702653087, 0.26705238102427886, 0.2393086769107607, 0.21690216474877644, 0.19834797338980364, 0.1826807859849803, 0.1692423884973554, 0.15756654773338755, 0.14731257622096056, 0.13822498800698133, 0.13010794307028975]


1

In [7]:
ΔE = 0 * LLT.W0                # Energy difference between the two layers

# Set up one-body Hamiltonian matrix for inter-layer tunneling
# H_onebody[c1, c2, cc, k] : no hopping from component c2 to c1
# For bilayer system: Nc_hopping=1, Nc_conserve=2, Nk=6
H_onebody_bilayer_no_tunneling = zeros(ComplexF64, 1, 1, Nc_conserve, Nk)
# Add energy difference terms: ΔE = 0.5 * W0 between different layers
# No tunneling
for k_idx in 1:Nk
    H_onebody_bilayer_no_tunneling[1, 1, 1, k_idx] = 0.5ΔE
    H_onebody_bilayer_no_tunneling[1, 1, 2, k_idx] = -0.5ΔE
end

# Create parameter structure for bilayer system
para_conserve = EDPara(
    k_list = k_list, 
    Gk = Gk, 
    Nc_hopping = 1,
    Nc_conserve = Nc_conserve,
    H_onebody = H_onebody_bilayer_no_tunneling,
    V_int = LLT.V_int_mixing_bilayer,      # use mixing interaction
    FF_inf_angle = LLT.Landau_ff_inf,
);

In [8]:
# Create momentum blocks for bilayer system
blocks, block_k1, block_k2, k0number = 
    ED_momentum_block_division(para_conserve, ED_mbslist(para_conserve, (Ne1,Ne2)));
length.(blocks)

12-element Vector{Int64}:
 71188
 71178
 71178
 71120
 71112
 71112
 71188
 71178
 71178
 71120
 71112
 71112

In [9]:
# Generate scattering lists for efficient Hamiltonian construction
scat_list1_conserve = ED_sortedScatteringList_onebody(para_conserve);
scat_list2_conserve = ED_sortedScatteringList_twobody(para_conserve);

In [ ]:
Neigen = 10  # Number of eigenvalues to compute per block
energies = Vector{Vector{Float64}}(undef, length(blocks))
vectors = Vector{Vector{Vector{ComplexF64}}}(undef, length(blocks))
for i in eachindex(blocks)
    println("Processing block #$i with size $(length(blocks[i])), momentum $(block_k1[i]), $(block_k2[i])")
    energies[i], vectors[i] = EDsolve(blocks[i], scat_list2_conserve, scat_list1_conserve;
        N = Neigen, showtime=true
    )
end

Processing block #1 with size 71188, momentum 0, 0
 18.082707 seconds (317.90 M allocations: 7.725 GiB, 15.91% gc time, 17.54% compilation time)
 15.285825 seconds (5.28 M allocations: 1006.209 MiB, 10.48% gc time, 53.17% compilation time)
Processing block #2 with size 71178, momentum 0, 1
 15.382439 seconds (316.83 M allocations: 7.673 GiB, 14.40% gc time)
  6.505100 seconds (5.07 k allocations: 644.621 MiB, 5.98% gc time)
Processing block #3 with size 71178, momentum 0, 2
 22.951228 seconds (316.83 M allocations: 7.673 GiB, 13.12% gc time)
 11.738236 seconds (5.07 k allocations: 644.622 MiB, 4.95% gc time)
Processing block #4 with size 71120, momentum 1, 0
 26.131134 seconds (316.52 M allocations: 7.665 GiB, 14.29% gc time)
  7.332210 seconds (6.64 k allocations: 869.969 MiB, 8.24% gc time)
Processing block #5 with size 71112, momentum 1, 1
 24.914715 seconds (316.47 M allocations: 7.663 GiB, 13.33% gc time)
  8.703384 seconds (6.31 k allocations: 809.073 MiB, 5.48% gc time)
Processi

In [ ]:
plot_ed_spectrum()